# Restaurant Recommendation Model

## Purpose
This Python notebook is dedicated to building and training the machine learning model used as the backend for the Restaurant Recommendation System. The goal is to create a robust recommendation system that suggests restaurants based on user preferences, historical data, and other relevant features.

## Frameworks Used
Built with sci.kit learn and XGBoost. The GradientBoostingRegressor was used to build this system.

In [2]:
#Import Relevant Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Read Pandas Dataframe
complete = pd.read_csv("./data/restaurantdemo.csv") #Directory
complete = complete.drop(columns=['business_id', 'GEO_ID', 'address', 'stars', 'review_count'])
print(complete.head())

                       name         city state  postal_code   latitude  \
0              Metro Grille     Flanders    NJ         7836  39.949904   
1  Helen's Cafe and Gardens      Alloway    NJ         8001  39.563830   
2       Alloway Village Inn      Alloway    NJ         8001  39.555717   
3                McDonald's  Cherry Hill    NJ         8002  39.936079   
4              Little Tokyo  Cherry Hill    NJ         8002  39.943728   

   longitude                                         categories  B01001_001E  \
0 -75.161599          Restaurants, Asian Fusion, American (New)        12034   
1 -75.363824  Restaurants, Cafes, Breakfast & Brunch, Venues...          835   
2 -75.360766  American (Traditional), Restaurants, Bars, Nig...          835   
3 -75.044117  Burgers, Food, Restaurants, Fast Food, Coffee ...        23630   
4 -75.026066                  Japanese, Sushi Bars, Restaurants        23630   

   B01001_002E  B01001_003E  ...  S1903_C03_032E  S1903_C03_033E  \
0     

/var/folders/6l/xb2bmr4d5718m4gjtrxd8h2w0000gn/T/ipykernel_39314/1322297738.py:2: DtypeWarning: Columns (116,117,123,124,125,127,128,129,130,132,133,139,140,145,146,147,149,150,153,154) have mixed types. Specify dtype option on import or set low_memory=False.
  complete = pd.read_csv("./data/restaurantdemo.csv") #Directory


In [4]:
#Sk.Learn Imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Apply TF-IDF to each text column separately
tfidf_vectorizer = TfidfVectorizer()

for col in ['name', 'city', 'state', 'categories']:
    tfidf_matrix = tfidf_vectorizer.fit_transform(complete[col])
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
    complete = pd.concat([complete, tfidf_df], axis=1)

# Drop the original text columns
complete = complete.drop(['name', 'city', 'state', 'categories'], axis=1)

In [6]:
print(complete.head())

   postal_code   latitude  longitude  B01001_001E  B01001_002E  B01001_003E  \
0         7836  39.949904 -75.161599        12034         5893          355   
1         8001  39.563830 -75.363824          835          420            0   
2         8001  39.555717 -75.360766          835          420            0   
3         8002  39.936079 -75.044117        23630        11866         1068   
4         8002  39.943728 -75.026066        23630        11866         1068   

   B01001_004E  B01001_005E  B01001_006E  B01001_007E  ...  wineries  wings  \
0          267          586           62          174  ...       0.0    0.0   
1            0           33            9            5  ...       0.0    0.0   
2            0           33            9            5  ...       0.0    0.0   
3          752          650          422          245  ...       0.0    0.0   
4          752          650          422          245  ...       0.0    0.0   

   women  wraps  yelp  yoga  yogurt  your  yoursel

In [7]:
import xgboost as xgb

In [8]:
# Normalize all columns
complete = complete.apply(pd.to_numeric, errors='coerce')

for col in complete.columns:
    col_min = complete[col].min()
    col_range = complete[col].max() - col_min
    complete[col] = (complete[col] - col_min) / col_range

print(complete.head())

   postal_code  latitude  longitude  B01001_001E  B01001_002E  B01001_003E  \
0     0.000000  0.764098   0.989054     0.120581     0.120460     0.086228   
1     0.001868  0.740280   0.984602     0.008367     0.008585     0.000000   
2     0.001868  0.739780   0.984669     0.008367     0.008585     0.000000   
3     0.001879  0.763246   0.991641     0.236774     0.242554     0.259412   
4     0.001879  0.763717   0.992038     0.236774     0.242554     0.259412   

   B01001_004E  B01001_005E  B01001_006E  B01001_007E  ...  wineries  wings  \
0     0.067732     0.148204     0.025931     0.038275  ...       0.0    0.0   
1     0.000000     0.008346     0.003764     0.001100  ...       0.0    0.0   
2     0.000000     0.008346     0.003764     0.001100  ...       0.0    0.0   
3     0.190766     0.164390     0.176495     0.053894  ...       0.0    0.0   
4     0.190766     0.164390     0.176495     0.053894  ...       0.0    0.0   

   women  wraps  yelp  yoga  yogurt  your  yourself  zoo

In [9]:
complete = complete.loc[:, ~complete.columns.duplicated()]

In [10]:
# Assume 'stars' is the column you want to predict
y = complete['star_count']

# Drop the target column from the features
X = complete.drop('star_count', axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model setup
bst = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Model training
bst.fit(X_train, y_train)

# Predictions on the test set
predictions = bst.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.010804586658382947


### Sk.Learn Gradient Boosting

In [12]:
#Train Test Split Setup
y = complete['stars']
X = complete.drop(columns=['stars'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Text Columns
text_features = ['name', 'city', 'state', 'categories']
numeric_features = X_train.select_dtypes(include=['number']).columns

Mean Squared Error: 0.43657840168681056


## BERT Testing via HuggingFace
Using batch processing to tokenize and find embeddings for each of the text-based columns.

In [4]:
from transformers import BertTokenizer, BertModel
import torch

/Users/mihir/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mihir/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Create
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [6]:
# Text Columns
text_column = ['name', 'city', 'state', 'categories']

# Batch Computations
batch_size = 100  # 100 Rows Per Call
num_batches = (len(complete) + batch_size - 1) // batch_size

# Store Embeddings
bert_embeddings_list = []

# Loop Through Each Batch and Store
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size

    # Tokenize
    batch_texts = list(complete[text_column].iloc[start_idx:end_idx])
    tokenized = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**tokenized)

    # Extract
    embeddings = outputs.last_hidden_state[:, 0, :]

    # Convert
    bert_embeddings_list.append(embeddings.numpy())

# Concatenate
concatenated_embeddings = np.concatenate(bert_embeddings_list, axis=0)

# New Columns
new_column_names = [f'{text_column}_bert_embedding_{i}' for i in range(embeddings.shape[1])]

# DF with Embeddings
bert_embeddings_df = pd.DataFrame(concatenated_embeddings, columns=new_column_names)

# Concatenate in PD
complete = pd.concat([complete, bert_embeddings_df], axis=1)

In [7]:
# Save Data with Embeddings
complete.to_csv('./data/embedded.csv', index=False)

### XGBoost Test

In [7]:
# Import
import xgboost as xgb

In [8]:
# Convert all object columns to numeric
for col in complete.select_dtypes(include='object').columns:
    complete[col] = pd.to_numeric(complete[col], errors='coerce')


complete = complete.drop(columns=['name', 'city', 'state'])
#complete.to_csv('./data/modelready.csv', index=False)

In [9]:
# Remove Invalid Column Names
original_columns = complete.columns
replacement_mapping = {col: col.replace('[', '_').replace(']', '_') for col in original_columns if '_bert_embedding_' in col}
complete.rename(columns=replacement_mapping, inplace=True)

print(complete.head())

   postal_code   latitude  longitude  stars  review_count  categories  \
0         7836  39.949904 -75.161599    3.0            17         NaN   
1         8001  39.563830 -75.363824    4.0             5         NaN   
2         8001  39.555717 -75.360766    3.5            13         NaN   
3         8002  39.936079 -75.044117    2.5            18         NaN   
4         8002  39.943728 -75.026066    4.0            12         NaN   

   B01001_001E  B01001_002E  B01001_003E  B01001_004E  ...  \
0        12034         5893          355          267  ...   
1          835          420            0            0  ...   
2          835          420            0            0  ...   
3        23630        11866         1068          752  ...   
4        23630        11866         1068          752  ...   

   _'name', 'city', 'state', 'categories'__bert_embedding_758  \
0                                           0.228715            
1                                           0.080919      

In [10]:
#Train Test Split Setup
y = complete['stars'] # just stars column
X = complete.drop(columns=['stars']) # df with no stars column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Parameters
params = {
    'objective': 'reg:squarederror',  # Regression task with squared error loss
    'learning_rate': 0.1,              # Step size shrinkage to prevent overfitting
    'max_depth': 2,                    # Maximum depth of a tree
    'subsample': 0.8,                  # Fraction of samples used for training each tree
    'colsample_bytree': 0.8,           # Fraction of features used for training each tree
    'n_estimators': 100,               # Number of boosting rounds (trees)
    'random_state': 42                  # Random seed for reproducibility
}

# Model
bst = xgb.XGBRegressor(**params)

# Fit
bst.fit(X_train, y_train)

# Predict
predictions = bst.predict(X_test)


In [26]:
# Assuming X_test, y_test are your testing data
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error on Test Set: {mse}')

Mean Squared Error on Test Set: 0.631971720180573


In [27]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.6343937692518058
